## 2018년 05월 18일 15시 09분 36초에 추가 ##
- Training FCN_RGB on NYUDv2

In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from SUNRGBDLoader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
data_path = '/home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/'
arg_string = '--arch FCN_RGB --batch_size 1 --n_epoch 50'

# Argument setting

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=10, help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None, help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)

# Model init

In [ ]:
args = parser.parse_args(arg_string.split(' '))

# Setup Dataloader
t_loader = SUNRGBDLoader(data_path, is_transform=True)
v_loader = SUNRGBDLoader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


In [ ]:
for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
    print(i_val, valloader.dataset.label_files['test'][i_val])

25it [00:00, 85.43it/s]

0 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000001.png
1 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000002.png
2 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000003.png
3 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000004.png
4 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000005.png
5 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000006.png
6 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000007.png
7 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000008.png
8 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000009.png
9 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000010.png
10 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000011.png
11 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000012.png
12 /home/dongwonshin/Deskt

65it [00:00, 129.65it/s]

47 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000048.png
48 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000049.png
49 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000050.png
50 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000051.png
51 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000052.png
52 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000053.png
53 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000054.png
54 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000055.png
55 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000056.png
56 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000057.png
57 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000058.png
58 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-000059.png
59 /home/dongwon

107it [00:00, 150.88it/s]

...

4871it [00:22, 215.20it/s]

4839 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004841.png
4840 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004842.png
4841 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004843.png
4842 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004844.png
4843 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004845.png
4844 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004846.png
4845 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004847.png
4846 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004848.png
4847 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004849.png
4848 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004850.png
4849 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004851.png
4850 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004

4911it [00:22, 214.89it/s]

4873 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004875.png
4874 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004876.png
4875 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004877.png
4876 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004878.png
4877 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004879.png
4878 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004880.png
4879 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004881.png
4880 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004882.png
4881 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004883.png
4882 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004884.png
4883 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004885.png
4884 /home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD/test/label/label-004

4934it [00:22, 214.93it/s]

In [ ]:
model.train()
for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
    color_imgs = Variable(color_imgs.cuda())
    depth_imgs = Variable(depth_imgs.cuda())
    label_imgs = Variable(label_imgs.cuda())

    optimizer.zero_grad()
    outputs = model(color_imgs)

    loss = loss_fn(input=outputs, target=label_imgs)

    loss.backward()
    optimizer.step()
    
    print(i)

In [ ]:
model.eval()
for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
    color_images_val = Variable(color_images_val.cuda(), volatile=True)
    depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
    label_images_val = Variable(label_images_val.cuda(), volatile=True)

    outputs = model(color_images_val)
    pred = outputs.data.max(1)[1].cpu().numpy()
    gt = label_images_val.data.cpu().numpy()
    running_metrics.update(gt, pred)

In [ ]:
print('hello')

In [ ]:
score, class_iou = running_metrics.get_scores()
for k, v in score.items():
    print(k, v)
running_metrics.reset()

if score['Mean IoU : \t'] >= best_iou:
    best_iou = score['Mean IoU : \t']
    state = {'epoch': epoch+1,
             'model_state': model.state_dict(),
             'optimizer_state' : optimizer.state_dict(),}
    torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'SUNRGBD'))

# Training

In [ ]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        color_imgs = Variable(color_imgs.cuda())
        depth_imgs = Variable(depth_imgs.cuda())
        label_imgs = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(color_imgs)

        loss = loss_fn(input=outputs, target=label_imgs)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 100 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'SUNRGBD'))